# Content Search Using Weighted Rank TF-IDF
### `Data Souce:-` I have scrapped data from https://clinicaltrials.gov/ website & lodaded into MySQL DB.
### [View Web Scraper & Load Data To MySQL ETL Notebook](https://nbviewer.org/github/pranabkumarpaul/Web_Scraper_And_ETL/blob/main/Web_Scraper_%26_Load_To_MySQL_ETL.ipynb)
<br><br>

## Install & Import Libraries

In [1]:
# !pip install nltk
# !pip3 install num2words==0.5.10
# import nltk
# nltk.download('omw-1.4')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import re
import math
import pickle
import pymysql
import numpy as np
import pandas as pd
from num2words import num2words
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
stop_words = stopwords.words('english')
pd.set_option('display.max_colwidth', None)

## User Input

In [5]:
### MySQL DB Info
Schema = "clinical_studies"
Table = "land_clinical_trials"
SQL_CLM = 'Row_No , NCT_Id , `Condition` , Study_Title , Study_Type'

MySQL_Endpoint = "127.0.0.1"
MySQL_Port = "3306"
MySQL_UserName = "root"
MySQL_Password = "MySql@1234"
MySQL_DataBase = "clinical_studies"

## Create MySQL Connection

In [6]:
MySQL_Connection = pymysql.connect(host= MySQL_Endpoint, 
                                   user= MySQL_UserName, 
                                   password= MySQL_Password, 
                                   db= MySQL_DataBase)

MySQL_Cursor = MySQL_Connection.cursor()

## Read Data From MySQL DB & Remove Duplicate Rows If Any

In [7]:
MySQL_Cursor.execute(f"Select {SQL_CLM} from {Schema}.{Table}")
SQL_Row = MySQL_Cursor.fetchall()

MySQL_Cursor.close()
MySQL_Connection.close()

Raw_Data = pd.DataFrame(SQL_Row, columns= SQL_CLM.split(","))
del SQL_Row

print(f"Shape before deleting duplicate values:- {Raw_Data.shape}")

Raw_Data.drop_duplicates()

print(f"Shape after deleting duplicate values:- {Raw_Data.shape}")

Raw_Data.head(3)

Shape before deleting duplicate values:- (20000, 5)
Shape after deleting duplicate values:- (20000, 5)


,Row_No,NCT_Id,`Condition`,Study_Title,Study_Type
0,1,NCT05516706,Sports Physical Therapy,Comparison of Dynamic Stretching and Plyometric Push up Training on Upper Body Performance Tests in Cricketers,Interventional
1,2,NCT05516693,"Mastication Disorder, Swallowing Disorder, Orofacial Dyskinesia","Chewing, Swallowing and Orofacial Motricity in Severe Obese",Interventional
2,3,NCT05516680,"Poststroke Depression, Healthy",Effects and Central Mechanism of Electroacupuncture and MRInavigated rTMS for PSD,Interventional


## Basic Data Exploration

In [8]:
Raw_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Row_No         20000 non-null  int64 
 1    NCT_Id        20000 non-null  object
 2    `Condition`   20000 non-null  object
 3    Study_Title   20000 non-null  object
 4    Study_Type    20000 non-null  object
dtypes: int64(1), object(4)
memory usage: 781.4+ KB


- ### Statistics

In [9]:
Raw_Data.describe(include = 'all')

,Row_No,NCT_Id,`Condition`,Study_Title,Study_Type
count,20000.000000,20000,20000,20000,20000
unique,NaN,19428,12841,19367,4
top,NaN,NCT05281497,Healthy,Trial of device that is not approved or cleared by the U.S. FDA,Interventional
freq,NaN,2,219,61,14950
mean,10000.500000,NaN,NaN,NaN,NaN
std,5773.647028,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN
25%,5000.750000,NaN,NaN,NaN,NaN
50%,10000.500000,NaN,NaN,NaN,NaN
75%,15000.250000,NaN,NaN,NaN,NaN


- ### Missing Values

In [10]:
Raw_Data.isnull().sum()

Row_No           0
 NCT_Id          0
 `Condition`     0
 Study_Title     0
 Study_Type      0
dtype: int64

- ### Columns Name

In [11]:
New_Column = []

for Each_Col in Raw_Data.columns:
    New_Column.append(Each_Col.strip().replace("`",""))
    
print(New_Column)

Raw_Data.columns = New_Column

Raw_Data.head(3)

['Row_No', 'NCT_Id', 'Condition', 'Study_Title', 'Study_Type']


,Row_No,NCT_Id,Condition,Study_Title,Study_Type
0,1,NCT05516706,Sports Physical Therapy,Comparison of Dynamic Stretching and Plyometric Push up Training on Upper Body Performance Tests in Cricketers,Interventional
1,2,NCT05516693,"Mastication Disorder, Swallowing Disorder, Orofacial Dyskinesia","Chewing, Swallowing and Orofacial Motricity in Severe Obese",Interventional
2,3,NCT05516680,"Poststroke Depression, Healthy",Effects and Central Mechanism of Electroacupuncture and MRInavigated rTMS for PSD,Interventional


## Saving The Data As A Pickle File

In [12]:
Raw_Data.to_pickle("Raw_Data.pkl")

## Total Length Of The DataFrame

In [13]:
Total_Row = len(Raw_Data)
Total_Row

20000

## UDF - Function To Process Data

In [14]:
def Data_Processor(Data_):
    import re
    
    ### To Lower Case
    Data_ = Data_.lower()
    
    ### Removing All The Special Characters or Punctuation
    Data_ = re.sub(r'[!|\|"|#|$|%|&|(|)|*|+|-|.|/|:|;|<|=|>|?|@|[|]|^|_|`|{|}|~|\n]' , r'' , Data_)
    
    ### Removing Stop Words & Lemmatisation
    stop_words_ = stopwords.words('english')
    Word_Lemmatizer = WordNetLemmatizer()
    
    All_Words = word_tokenize(Data_)
    
    New_Text = ""
    for Each_Word in All_Words:
        if Each_Word not in stop_words_ and len(Each_Word) > 1:
            Lemmatize_Word = Word_Lemmatizer.lemmatize(Each_Word)
            New_Text = New_Text + " " + Lemmatize_Word
            
    ### Remove Apostrophe & RETURN
    return New_Text.replace("'","")

## Extracting Informative Text

In [15]:
Processed_Condition = []
Processed_Study_Title = []

for Each_Row in range(Total_Row):
    Processed_Condition.append(word_tokenize(str(Data_Processor(Data_=Raw_Data['Condition'][Each_Row]))))
    Processed_Study_Title.append(word_tokenize(str(Data_Processor(Data_=Raw_Data['Study_Title'][Each_Row]))))

### Last Row Number
print(f"Last Row Number- {Each_Row}\n")
print(f"Condition Column - \n{Processed_Condition[Each_Row]}\n")
print(f"Study_Title Column - \n{Processed_Study_Title[Each_Row]}")

Last Row Number- 19999

Condition Column - 
['addiction', 'opiate']

Study_Title Column - 
['comparison', 'different', 'psychotherapy', 'intervention', 'regarding', 'effect', 'substance', 'craving']


## Calculating Occurrence Of The Word In Entire Document
- #### Entire Document = Processed_Condition + Processed_Study_Title

In [16]:
Word_Frequency_In_Entire_Document = {}

for Each_Row in range(Total_Row):
    
    for Each_Word in Processed_Condition[Each_Row]:
        
        if Each_Word not in Word_Frequency_In_Entire_Document.keys():
            Word_Frequency_In_Entire_Document[Each_Word] = 1
        else:
            Word_Frequency_In_Entire_Document[Each_Word] += 1
            
    for Each_Word in Processed_Study_Title[Each_Row]:
        
        if Each_Word not in Word_Frequency_In_Entire_Document.keys():
            Word_Frequency_In_Entire_Document[Each_Word] = 1
        else:
            Word_Frequency_In_Entire_Document[Each_Word] += 1
            
Total_Vocabulary_In_Entire_Document = len(Word_Frequency_In_Entire_Document)
print(f"Total Words In Word_Frequency_In_Entire_Document Vocabulary:- {Total_Vocabulary_In_Entire_Document}\n")

Total Words In Word_Frequency_In_Entire_Document Vocabulary:- 23619



## Sample Words In Vocabulary

In [17]:
Vocabulary_Words = [key for key in Word_Frequency_In_Entire_Document.keys()]
print(Vocabulary_Words[:5])
print(Vocabulary_Words[1000:1005])

['sport', 'physical', 'therapy', 'comparison', 'dynamic']
['optic', 'thc', 'neuroinflammatory', 'cannabidiol', 'tetrahydrocannabinol']


## Occurrence Of A Sample Word - `"cancer"`

In [18]:
Word_Frequency_In_Entire_Document['cancer']

4503

## Calculating TF-IDF For Condition

In [19]:
Doc_No = 0

TF_IDF_Condition = {}

for Each_Row in range(Total_Row):
    
    Words = Processed_Condition[Each_Row]
    Counter_ = Counter(Words)
    Words_Count = len(Words)
    

    for Each_word in np.unique(Words):
        
        TF = Counter_[Each_word] / Words_Count
        Word_Frequency = Word_Frequency_In_Entire_Document[Each_word]
        IDF = np.log( ( Total_Row + 1 ) / ( Word_Frequency + 1 ) ) # Numerator Is Added 1 To Avoid Negative Values
        
        TF_IDF_Condition[Doc_No , Each_word] = TF * IDF
        
    Doc_No += 1
    
print(f"Sample Word & TF-IDF Value - \n\n{dict(list(TF_IDF_Condition.items())[0: 5])}")    

Sample Word & TF-IDF Value - 

{(0, 'physical'): 1.3419339231688436, (0, 'sport'): 1.9363976630213564, (0, 'therapy'): 0.925816759329015, (1, 'disorder'): 0.8550998004068601, (1, 'dyskinesia'): 1.2509403797479663}


## Calculating TF-IDF For Study_Title

In [20]:
Doc_No = 0

TF_IDF_Study_Title = {}

for Each_Row in range(Total_Row):
    
    Words = Processed_Study_Title[Each_Row]
    Counter_ = Counter(Words)
    Words_Count = len(Words)
    
    for Each_word in np.unique(Words):
        
        TF = Counter_[Each_word] / Words_Count
        Word_Frequency = Word_Frequency_In_Entire_Document[Each_word]
        IDF = np.log( ( Total_Row + 1 ) / ( Word_Frequency + 1 ) ) # Numerator Is Added 1 To Avoid Negative Values
        
        TF_IDF_Study_Title[Doc_No , Each_word] = TF * IDF
        
    Doc_No += 1
    
print(f"Sample Word & TF-IDF Value - \n\n{dict(list(TF_IDF_Study_Title.items())[0: 5])}")     

Sample Word & TF-IDF Value - 

{(0, 'body'): 0.41378903798273664, (0, 'comparison'): 0.3637185652622549, (0, 'cricketer'): 0.8373082155205659, (0, 'dynamic'): 0.5563043561243554, (0, 'performance'): 0.39003722226346355}


## Merging Both TF-IDF (TF_IDF_Condition , TF_IDF_Study_Title) By Weights

In [21]:
Weights = 0.40

- #### Multiply The Entire TF_IDF_Condition With The Weights

In [22]:
for i in TF_IDF_Condition:
    TF_IDF_Condition[i] *= Weights
    
print(f"Sample Word & TF-IDF Value - \n\n{dict(list(TF_IDF_Condition.items())[0: 5])}")        

Sample Word & TF-IDF Value - 

{(0, 'physical'): 0.5367735692675375, (0, 'sport'): 0.7745590652085426, (0, 'therapy'): 0.37032670373160603, (1, 'disorder'): 0.34203992016274404, (1, 'dyskinesia'): 0.5003761518991866}


- #### Iterate The Words in Study_Title & Replace The Value Of Condition

In [23]:
for i in TF_IDF_Study_Title:
    TF_IDF_Condition[i] = TF_IDF_Study_Title[i]

## Saving The Dictionary As A Pickle File

In [24]:
with open ("TF_IDF_Condition_PKL.pkl", "wb") as TF_IDF_S_C_PKL:
    pickle.dump(TF_IDF_Condition, TF_IDF_S_C_PKL , protocol= pickle.HIGHEST_PROTOCOL)
TF_IDF_S_C_PKL.close()

# <br><br>UDF - Final Search Function

In [25]:
def Best_Match_Records(String_To_Search , Records_To_Show = 5):
    
    import re
    import pickle
    import pandas as pd
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer
    
    stop_words = stopwords.words('english')
    
    Raw_Data = pd.read_pickle("Raw_Data.pkl")
    
    with open ("TF_IDF_Condition_PKL.pkl" , "rb") as TF_IDF_S_C:
        TF_IDF_Condition = pickle.load(TF_IDF_S_C)
        TF_IDF_S_C.close()

###### Function To Process Data
    def Data_Processor(Data_):

        ### To Lower Case
        Data_ = Data_.lower()

        ### Removing All The Special Characters or Punctuation
        Data_ = re.sub(r'[!|\|"|#|$|%|&|(|)|*|+|-|.|/|:|;|<|=|>|?|@|[|]|^|_|`|{|}|~|\n]' , r'' , Data_)

        ### Removing Stop Words & Lemmatisation
        stop_words_ = stopwords.words('english')
        Word_Lemmatizer = WordNetLemmatizer()

        All_Words = word_tokenize(Data_)

        New_Text = ""
        for Each_Word in All_Words:
            if Each_Word not in stop_words_ and len(Each_Word) > 1:
                Lemmatize_Word = Word_Lemmatizer.lemmatize(Each_Word)
                New_Text = New_Text + " " + Lemmatize_Word

        ### Remove Apostrophe & RETURN
        return New_Text.replace("'","")
        
#######################################################################################    
        
    Search_Words_Token = word_tokenize(str(Data_Processor(String_To_Search)))
    
    print(f"Search String:- \n{String_To_Search}\n")
    print(f"Search String Words Token:- \n{Search_Words_Token}")
    
    Query_Weights = {}
    
    for KEY in TF_IDF_Condition:
        
        if KEY[1] in Search_Words_Token:
            try:
                Query_Weights[KEY[0]] += TF_IDF_Condition[KEY]
            except:
                Query_Weights[KEY[0]] = TF_IDF_Condition[KEY]
                
    Query_Weights = sorted(Query_Weights.items() , key = lambda x: x[1] , reverse = True)
    
    Match_Records = []
    
    for No in Query_Weights[ : Records_To_Show]:
        Match_Records.append(No[0])
        
    print(f"\nRow No That Has Been Matched:- \n{Match_Records}")
    display(Raw_Data.loc[ Match_Records , ])

# Call Search Function

In [26]:
Best_Match_Records("cancer bleeding")

Search String:- 
cancer bleeding

Search String Words Token:- 
['cancer', 'bleeding']

Row No That Has Been Matched:- 
[17506, 8723, 6301, 812, 3878]


,Row_No,NCT_Id,Condition,Study_Title,Study_Type
17506,17507,NCT05287048,Endometrial Cancer,Evaluation of MCM5 in Postmenopausal Bleeding Patients,Observational
8723,8724,NCT05403203,Placenta Previa Bleeding,Prediction of Bleeding in Placenta Previa,Observational
6301,6302,NCT05434728,"EhlersDanlos Syndrome, EDS, Classical EhlersDanlos Syndrome, Classical EDS cEDS, Hypermobile EhlersDanlos Syndrome, Hypermobile EDS hEDS, Vascular EhlersDanlos Syndrome, Vascular EDS vEDS",Characterization of Bleeding Disorders in EDS,Observational
812,813,NCT05506150,"Gastro Intestinal Bleeding, Patient Engagement, Family Members",Patient Important Gastrointestinal Bleeding in the ICU,Observational
3878,3879,NCT05464394,"Hemorrhage, Bleeding",Peroperative Administration of Tranexamic Acid in RouxenY Gastric Bypass and Oneanastomosis Gastric Bypass,Interventional
